<a href="https://colab.research.google.com/github/YN1UJE/repo/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 9.7 MB/s eta 0:00:00


In [5]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('ngrok_authtoken'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage, StickerMessage, ImageMessage, VideoMessage, LocationMessage
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('line_channel_access_token'))
handler = WebhookHandler(userdata.get('line_channel_secret'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


import google.generativeai as genai

genai.configure(api_key=userdata.get('api_key'))
model = genai.GenerativeModel("gemini-2.0-flash-exp")
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == "sticker":
          #回覆貼圖
          reply = StickerMessage(
              package_id='6136',
              sticker_id='10551379'
          )
        elif action == "image":
          #回覆圖片
          reply = ImageMessage(
            original_content_url='https://assets.papegames.com/nikkiweb/papegame/deepspaceen/material/20250127/e0a0a17ce6b993abb01d620a2f05124f.jpg',
            preview_image_url='https://assets.papegames.com/nikkiweb/papegame/deepspaceen/material/20250127/e0a0a17ce6b993abb01d620a2f05124f.jpg'
          )
        elif action == "video":
          #回覆影片
          reply = VideoMessage(
            original_content_url='https://assets.papegames.com/nikkiweb/papegame/deepspacetw/material/20250110/659cb37219fed672bed74a97cd3d8923.mp4',
            preview_image_url='https://assets.papegames.com/nikkiweb/papegame/deepspacetw/material/20250110/c97a766be012aa21cefb0eb16334b435.jpg'
          )
        elif action == "location":
          #回覆地點
          reply = LocationMessage(
            title="安利美特 高雄店",
            address="807高雄市三民區建國二路320號B3",
            latitude=22.639326587540392,
            longitude=120.30259179502805
          )
        else:
          response = ask_gemini(action)
          reply = TextMessage(text=response)

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://9647-34-57-236-17.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [20/May/2025 07:20:00] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/May/2025 07:20:00] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/May/2025 07:20:13] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/May/2025 07:20:23] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/May/2025 07:20:27] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/May/2025 07:21:11] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/May/2025 07:22:11] "POST /callback HTTP/1.1" 200 -
